In [21]:
base_url = "https://raw-data-api0.hotosm.org/v1"
headers = {"accept": "application/json", "Content-Type": "application/json"}

In [33]:
query_url = f"{base_url}/snapshot/"
query_url

'https://raw-data-api0.hotosm.org/v1/snapshot/'

In [118]:
import requests
import json
import zipfile
from io import BytesIO
import numpy as np
import pandas as pd

In [ ]:
pip install folium

In [ ]:
import folium

In [164]:
m = folium.Map(location=[27.6866, 83.4323], 
                 zoom_start=10, control_scale=True)

In [152]:
query={"geometry": {"type": "Polygon", "coordinates": [[[85.314267262, 27.708944912], [85.304948177, 27.695326815], [85.332592395, 27.694783741], [85.334081941, 27.711423854], [85.314267262, 27.708944912]]]}, "filters": {"tags": {"all_geometry": {"join_or": {"amenity": ["college", "indergarten", "school", "music_school", "language_school", "university"]}}}}, "geometryType": ["point", "polygon"], "centroid": "true"}

In [163]:
geom = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              85.31426726223691,
              27.70894491154847
            ],
            [
              85.30494817745944,
              27.695326814805895
            ],
            [
              85.33259239548863,
              27.694783740754275
            ],
            [
              85.33408194130885,
              27.711423854043616
            ],
            [
              85.31426726223691,
              27.70894491154847
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [165]:
m.add_child(folium.GeoJson(data = geom))

In [145]:
result = requests.post(query_url, data=json.dumps(query), headers=headers)
result.status_code

200

In [146]:
if result.status_code == 200:
    task_id = result.json()['task_id']
    task_id

In [147]:
task_url = f"{base_url}/tasks/status/{task_id}"
# extracts = requests.get(task_url)
while True:
    result = requests.get(task_url, headers=headers)
    if result.json()['status'] == "PENDING":
        time.sleep(1)
    elif result.json()['status'] == "SUCCESS":
        break
extracts.json()

{'id': '1ae562cd-f0d3-4006-97f9-1ae721c27dc5',
 'status': 'SUCCESS',
 'result': {'download_url': 'https://s3.us-east-1.amazonaws.com/exports-stage.hotosm.org/Export_geojson_uid_1ae562cd-f0d3-4006-97f9-1ae721c27dc5.zip',
  'file_name': 'Export',
  'process_time': '0:00:01.198945',
  'query_area': '0.06 Sq Km',
  'binded_file_size': '0.01 MB',
  'zip_file_size_bytes': 1433}}

In [151]:
zip_url = extracts.json()['result']['download_url']
zip_url
result = requests.get(zip_url, headers=headers)
# result.content
fp = BytesIO(result.content)
zfp = zipfile.ZipFile(fp, "r")
zfp.extract("Export.geojson", "/tmp/")
data = eval(zfp.read("Export.geojson"))


In [80]:
import pandas as pd
import numpy as np

In [140]:
m.add_child(folium.GeoJson(data = data))

In [132]:
center_point = [27, 83]

data = (
    np.random.normal(size=(100, 2)) *
    np.array([[.5, .5]]) +
    np.array([center_point])
)

df = pd.DataFrame(data, columns=['Lat', 'Long'])

df['Lat']

0     26.912726
1     26.610674
2     27.130600
3     27.109567
4     27.615503
        ...    
95    26.844529
96    27.303971
97    28.040139
98    27.048376
99    26.543785
Name: Lat, Length: 100, dtype: float64

In [123]:
m = folium.Map(df[['Lat', 'Long']].mean().values.tolist())

for lat, lon in zip(df['Lat'], df['Long']):
    folium.Marker([lat, lon]).add_to(m)

TypeError: 'str' object is not callable